In [28]:
!python -V

Python 3.9.12


In [29]:
!mlflow --version

mlflow, version 2.13.2


In [30]:
!pip install pyarrow

In [2]:
import pandas as pd
# df=pd.read_parquet('./homework/data/green_tripdata_2023-02.parquet')
# df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,1,2023-02-01 00:46:22,2023-02-01 01:05:57,N,1.0,74,265,1.0,10.80,42.9,1.0,1.5,0.00,0.0,NaN,1.0,45.40,2.0,1.0,0.00
1,2,2023-02-01 00:05:09,2023-02-01 00:22:42,N,1.0,216,196,1.0,4.76,23.3,1.0,0.5,0.00,0.0,NaN,1.0,25.80,2.0,1.0,0.00
2,2,2023-02-01 00:03:47,2023-02-01 00:27:30,N,1.0,7,114,1.0,6.32,30.3,1.0,0.5,8.89,0.0,NaN,1.0,44.44,1.0,1.0,2.75
3,2,2023-01-31 23:30:56,2023-01-31 23:51:40,N,1.0,74,239,1.0,3.50,16.3,1.0,0.5,0.00,0.0,NaN,1.0,21.55,2.0,1.0,2.75
4,2,2023-02-01 00:15:05,2023-02-01 00:26:02,N,1.0,82,223,1.0,3.14,17.0,1.0,0.5,0.00,0.0,NaN,1.0,19.50,2.0,1.0,0.00


In [123]:
#df[df.total_amount>400]
# df[df.trip_distance>100000]
# df.trip_distance.min()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
18474,2,2023-01-10 15:58:15,2023-01-10 15:58:26,N,5.0,92,92,1.0,0.38,440.0,0.0,0.0,0.0,0.0,None,1.0,441.0,2.0,2.0,0.0
21547,2,2023-01-11 20:52:23,2023-01-11 20:52:33,N,5.0,130,130,1.0,0.00,490.0,0.0,0.0,0.0,0.0,None,1.0,491.0,2.0,2.0,0.0
30047,2,2023-01-15 21:57:15,2023-01-15 21:57:20,N,5.0,191,191,1.0,0.00,450.0,0.0,0.0,0.0,0.0,None,1.0,451.0,2.0,2.0,0.0
42278,2,2023-01-21 19:47:15,2023-01-21 19:51:20,N,5.0,92,92,1.0,0.00,450.0,0.0,0.0,0.0,0.0,None,1.0,451.0,2.0,2.0,0.0
46373,2,2023-01-23 20:43:48,2023-01-23 20:43:53,N,5.0,130,130,1.0,0.00,459.9,0.0,0.0,0.0,0.0,None,1.0,460.9,2.0,2.0,0.0
50526,2,2023-01-25 17:14:16,2023-01-25 17:21:43,N,5.0,80,37,1.0,0.60,425.0,0.0,0.0,0.0,0.0,None,1.0,426.0,2.0,2.0,0.0
53670,2,2023-01-26 21:26:12,2023-01-26 21:26:17,N,5.0,129,129,1.0,0.00,428.0,0.0,0.0,0.0,0.0,None,1.0,429.0,2.0,2.0,0.0
59442,2,2023-01-29 20:01:58,2023-01-29 20:05:25,N,5.0,92,92,1.0,0.07,455.0,0.0,0.0,0.0,0.0,None,1.0,456.0,2.0,2.0,0.0


In [8]:
# df['duration_seconds']=(df['lpep_dropoff_datetime']-df['lpep_pickup_datetime']).dt.total_seconds()
# df['duration_seconds']

In [9]:
#df[['duration_seconds', 'trip_distance', 'total_amount']]

In [10]:
import pickle

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [13]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi1")

<Experiment: artifact_location='/workspaces/mlops-homeworks/2-Experiment-Tracking/mlruns/1', creation_time=1717680997723, experiment_id='1', last_update_time=1717680997723, lifecycle_stage='active', name='nyc-taxi1', tags={}>

In [18]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [30]:
df_train = read_dataframe('./homework/data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('./homework/data/green_tripdata_2023-02.parquet')

In [31]:
len(df_train), len(df_val)

(65946, 62574)

In [32]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [33]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [34]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [36]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.033419510656907

In [38]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [39]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Waleed")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2023-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [45]:
import xgboost as xgb

In [46]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [47]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [48]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [49]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                                                        | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:34:59] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.98719                                                                                                                         
[1]	validation-rmse:7.07177                                                                                                                         
[2]	validation-rmse:6.46555                                                                                                                         
[3]	validation-rmse:6.06169                                                                                                                         
[4]	validation-rmse:5.79865                                                                                                                         
[5]	validation-rmse:5.63231                                                                                                                         
[6]	validation-rmse:5.52171                                                                               

[54]	validation-rmse:5.23344                                                                                                                        
[55]	validation-rmse:5.23254                                                                                                                        
[56]	validation-rmse:5.23170                                                                                                                        
[57]	validation-rmse:5.23213                                                                                                                        
[58]	validation-rmse:5.23118                                                                                                                        
[59]	validation-rmse:5.23030                                                                                                                        
[60]	validation-rmse:5.22959                                                                              

[108]	validation-rmse:5.19253                                                                                                                       
[109]	validation-rmse:5.19209                                                                                                                       
[110]	validation-rmse:5.19159                                                                                                                       
[111]	validation-rmse:5.19097                                                                                                                       
[112]	validation-rmse:5.19052                                                                                                                       
[113]	validation-rmse:5.19095                                                                                                                       
[114]	validation-rmse:5.19065                                                                             

...

[54]	validation-rmse:5.22250                                                                                                                        
[55]	validation-rmse:5.22046                                                                                                                        
[56]	validation-rmse:5.21825                                                                                                                        
[57]	validation-rmse:5.21629                                                                                                                        
[58]	validation-rmse:5.21445                                                                                                                        
[59]	validation-rmse:5.21230                                                                                                                        
[60]	validation-rmse:5.21060                                                                              

[108]	validation-rmse:5.17534                                                                                                                       
[109]	validation-rmse:5.17527                                                                                                                       
[110]	validation-rmse:5.17531                                                                                                                       
[111]	validation-rmse:5.17500                                                                                                                       
[112]	validation-rmse:5.17465                                                                                                                       
[113]	validation-rmse:5.17445                                                                                                                       
[114]	validation-rmse:5.17371                                                                             

[162]	validation-rmse:5.16890                                                                                                                       
[163]	validation-rmse:5.16912                                                                                                                       
[164]	validation-rmse:5.16897                                                                                                                       
[165]	validation-rmse:5.16890                                                                                                                       
[166]	validation-rmse:5.16865                                                                                                                       
[167]	validation-rmse:5.16844                                                                                                                       
[168]	validation-rmse:5.16846                                                                             

[216]	validation-rmse:5.16188                                                                                                                       
[217]	validation-rmse:5.16166                                                                                                                       
[218]	validation-rmse:5.16164                                                                                                                       
[219]	validation-rmse:5.16143                                                                                                                       
[220]	validation-rmse:5.16143                                                                                                                       
[221]	validation-rmse:5.16123                                                                                                                       
[222]	validation-rmse:5.16119                                                                             

KeyboardInterrupt: 

In [50]:
mlflow.xgboost.autolog(disable=True)

In [52]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:44:40] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.73788
[1]	validation-rmse:8.22960
[2]	validation-rmse:7.78914
[3]	validation-rmse:7.40823
[4]	validation-rmse:7.08398
[5]	validation-rmse:6.80130
[6]	validation-rmse:6.56559
[7]	validation-rmse:6.35942
[8]	validation-rmse:6.18716
[9]	validation-rmse:6.04364
[10]	validation-rmse:5.91994
[11]	validation-rmse:5.81441
[12]	validation-rmse:5.72701
[13]	validation-rmse:5.65236
[14]	validation-rmse:5.58821
[15]	validation-rmse:5.53629
[16]	validation-rmse:5.49451
[17]	validation-rmse:5.45443
[18]	validation-rmse:5.42147
[19]	validation-rmse:5.39347
[20]	validation-rmse:5.37267
[21]	validation-rmse:5.35128
[22]	validation-rmse:5.33257
[23]	validation-rmse:5.31780
[24]	validation-rmse:5.30449
[25]	validation-rmse:5.29542
[26]	validation-rmse:5.28365
[27]	validation-rmse:5.27505
[28]	validation-rmse:5.26725
[29]	validation-rmse:5.26004
[30]	validation-rmse:5.25388
[31]	validation-rmse:5.24812
[32]	validation-rmse:5.24406
[33]	validation-rmse:5.24068
[34]	validation-rmse:5.2

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:45:06] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/06/06 15:50:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
